<a href="https://colab.research.google.com/gist/ruvnet/89fe2de3afa9bb9ddd3bf3400e21662a/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# D.R.E.A.M. AI System (Multimodal + Symbolic + Recursive Self-Optimization)
### created by rUv, cause he could.

This notebook demonstrates the **D.R.E.A.M. AI** system which integrates:

- **D**ual/Distributed Reasoning (multimodal: text, image, sensor)
- **R**ecursive Self-Optimization ("dreaming" cycles that fine‑tune the model)
- **E**mbodied & Environmentally Grounded (integrates sensor data)
- **A**daptive Multimodal Architecture (unified chain‑of‑thought with text + images)
- **M**ultimodal Visualization-of-Thought (MVoT) for enhanced interpretability

The system uses a transformer model (e.g. Chameleon‑7B) that generates a chain‑of‑thought, then applies a guided symbolic optimizer (GSPO) for subproblem solving (e.g. using A* for maze planning), and finally refines its reasoning via recursive self‑optimization. An evaluator CLI is provided so you can run evaluations directly from the command line, and a Gradio UI is included for interactive exploration.

# Installation & Setup

Run the cell below to install all required dependencies and set up the environment.

In [1]:
!apt-get -y install git-lfs > /dev/null 2>&1
!pip install torch transformers accelerate sympy gradio gymnasium numpy > /dev/null 2>&1

# Enable git-lfs
!git lfs install

# Clone the model repository (if needed; adjust URL as appropriate)
!git clone https://huggingface.co/facebook/chameleon-7b 2>/dev/null || echo "Model repo already cloned."

# Download example datasets for maze and behavior tasks
!mkdir -p data
!wget -q -O data/maze_dataset.zip https://example.com/maze_dataset.zip && unzip -q data/maze_dataset.zip -d data/
!wget -q -O data/minibehavior_dataset.zip https://example.com/minibehavior_dataset.zip && unzip -q data/minibehavior_dataset.zip -d data/
!pip install --upgrade transformers

print("Dependencies and datasets installed. Ready to proceed.")

Git LFS initialized.
Model repo already cloned.
Dependencies and datasets installed. Ready to proceed.


# Model & Tokenizer Setup

We load a multimodal transformer (Chameleon-7B) that supports both text and image tokens. Mixed-precision is enabled for efficient GPU usage.

In [34]:
from google.colab import userdata
import os
import torch
from transformers import AutoTokenizer, AutoModelForVision2Seq

# Retrieve the Hugging Face API key from Colab secrets
hf_api_key = userdata.get('HUGGINGFACE_API_KEY')
if not hf_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not found. Please add it to your Colab secrets.")

# Optionally, set the API key as an environment variable for later use
os.environ["HF_TOKEN"] = hf_api_key

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "facebook/chameleon-7b"  # Make sure you have accepted the model license and have access

print("Loading model. This may take a few minutes...")

model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_api_key,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=hf_api_key,
    trust_remote_code=True
)

model.eval()
print("Model and tokenizer loaded on", device)


Loading model. This may take a few minutes...


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1536: UserWarning: Current model requires 4096 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model and tokenizer loaded on cuda


# Token Discrepancy Loss (TDL)

This cell implements the token discrepancy loss used to align predicted image token embeddings with target embeddings.

In [2]:
import torch.nn as nn

# Example predicted and target image tokens
pred_image_tokens = torch.tensor([[1, 5, 9, 9, 2]], dtype=torch.long, device=device)
target_image_tokens = torch.tensor([[1, 7, 8, 2, 2]], dtype=torch.long, device=device)

embedding_matrix = model.get_input_embeddings()
pred_embeds = embedding_matrix(pred_image_tokens)
target_embeds = embedding_matrix(target_image_tokens)

tdl_loss_fn = nn.MSELoss()
tdl_loss = tdl_loss_fn(pred_embeds, target_embeds)

print("Token Discrepancy Loss (example):", tdl_loss.item())

Token Discrepancy Loss (example): 3.612041473388672e-05


# Multimodal Visualization-of-Thought (MVoT)

This cell implements a function to generate a chain‑of‑thought that interleaves text (and optionally image) tokens. Here we use a simple greedy decoding for demonstration.

In [3]:
def generate_mot_trace(prompt, max_new_tokens=100):
    """
    Generates a chain-of-thought given a prompt using the loaded model.
    Returns the decoded text (chain-of-thought).
    """
    input_ids = tokenizer(prompt, return_tensors='pt').to(device)
    generated = input_ids["input_ids"].clone()

    model.eval()
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(input_ids=generated)
            next_token = torch.argmax(outputs.logits[:, -1, :], dim=-1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

    chain_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    return chain_text

# Test the MVoT function
test_prompt = "Explain how you would solve a 5x5 maze with the entrance at top-left and exit at bottom-right."
chain_of_thought = generate_mot_trace(test_prompt, max_new_tokens=50)
print("Generated chain-of-thought:\n", chain_of_thought)

Generated chain-of-thought:
 Explain how you would solve a 5x5 maze with the entrance at top-left and exit at bottom-right.

To solve a 5x5 maze with the entrance at top-left and exit at bottom-right, we can use a systematic approach. Here's one possible solution:

1. Start at the top-left corner and


# Guided Symbolic Problem Optimization (GSPO)

Below is a complete implementation of a symbolic optimizer using A* search for maze path planning. This function finds an optimal path through a grid maze.

In [5]:
import heapq

def find_path_with_astar(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    def neighbors(pos):
        x, y = pos
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] == 0:
                yield (nx, ny)

    def heuristic(p, q):
        return abs(p[0]-q[0]) + abs(p[1]-q[1])

    open_set = [(heuristic(start, goal), 0, start, None)]
    came_from = {}
    cost_so_far = {start: 0}

    while open_set:
        _, cost, current, parent = heapq.heappop(open_set)
        if current in came_from:
            continue
        came_from[current] = parent
        if current == goal:
            break
        for nxt in neighbors(current):
            new_cost = cost + 1
            if nxt not in cost_so_far or new_cost < cost_so_far[nxt]:
                cost_so_far[nxt] = new_cost
                priority = new_cost + heuristic(nxt, goal)
                heapq.heappush(open_set, (priority, new_cost, nxt, current))

    path = []
    if goal in came_from:
        cur = goal
        while cur is not None:
            path.append(cur)
            cur = came_from[cur]
        path.reverse()
    return path

def is_path_valid(grid, path):
    return all(0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == 0 for (x, y) in path)

# Maze demonstration
maze_grid = [
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [1, 0, 1, 0],
    [0, 0, 0, 0]
]
start = (0, 0)
goal = (3, 3)

proposed_path = [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (3, 3)]

if not is_path_valid(maze_grid, proposed_path):
    print("GSPO: Proposed path is invalid. Running A* to find an alternative...")
    correct_path = find_path_with_astar(maze_grid, start, goal)
    print("GSPO: Suggested alternative path:", correct_path)
else:
    print("GSPO: Proposed path is valid.")

GSPO: Proposed path is invalid. Running A* to find an alternative...
GSPO: Suggested alternative path: [(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3)]


# Recursive Self-Optimization (Dreaming)

This cell implements a complete recursive self‑optimization loop. The model is prompted to generate a new problem (a "dream"), attempts to solve it, and is then fine‑tuned on a correct solution if the attempt fails. Here, we simulate evaluation by a random success indicator.

In [ ]:
import random
import time
import torch.nn as nn

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

def dream_and_improve(iterations=3):
    for i in range(iterations):
        print("\n====================")
        print(f"Iteration {i}: Starting dream and improvement cycle.")
        print("====================\n")

        # Step 1: Generate a new problem (dream) using a meta-prompt
        dream_prompt = "# Create a new challenging puzzle for the AI to solve."
        print("Generating dream problem with prompt:")
        print(dream_prompt)
        dream_input = tokenizer(dream_prompt, return_tensors='pt').to(device)
        with torch.no_grad():
            print("Running model.generate for dream problem...")
            dream_output = model.generate(**dream_input, max_new_tokens=64)
        dream_text = tokenizer.decode(dream_output[0], skip_special_tokens=True)
        print(f"\nIteration {i} - Dreamed Problem:\n{dream_text}")

        # Step 2: Solve the dreamed problem with chain-of-thought
        print("\nGenerating chain-of-thought based on the dream problem...")
        reasoning = generate_mot_trace(dream_text, max_new_tokens=64)
        print("\nGenerated Chain-of-Thought:\n", reasoning)

        # Step 3: Evaluate the solution (simulate evaluation)
        success = random.choice([True, False])
        if not success:
            print("\nEvaluation: The dreamed solution was incorrect. Initiating fine-tuning...")
            correct_solution = ("The correct reasoning is: first analyze the problem, then use logical deduction "
                                "and A* for path planning, finally output the optimized result.")
            print("Correct solution text:")
            print(correct_solution)

            # Determine a common max_length for both the dream text and correct solution
            input_ids_dream = tokenizer(dream_text)["input_ids"]
            input_ids_solution = tokenizer(correct_solution)["input_ids"]
            len_dream = len(input_ids_dream)
            len_solution = len(input_ids_solution)
            max_length = max(len_dream, len_solution)
            print(f"Token lengths - Dream: {len_dream}, Correct solution: {len_solution}. Using max_length = {max_length}")

            # Tokenize with padding and truncation to the same max_length
            inputs = tokenizer(dream_text, return_tensors='pt', padding="max_length",
                               max_length=max_length, truncation=True).to(device)
            labels = tokenizer(correct_solution, return_tensors='pt', padding="max_length",
                               max_length=max_length, truncation=True).to(device)
            print("Inputs shape:", inputs["input_ids"].shape)
            print("Labels shape before adjustment:", labels["input_ids"].shape)
            # Replace pad token ids in labels with -100 so they are ignored in loss computation
            labels["input_ids"][labels["input_ids"] == tokenizer.pad_token_id] = -100
            print("Labels shape after adjustment:", labels["input_ids"].shape)

            model.train()
            start_time = time.time()
            outputs = model(**inputs, labels=labels["input_ids"])
            loss = outputs.loss
            elapsed_time = time.time() - start_time
            print(f"Training step completed in {elapsed_time:.2f} seconds. Loss: {loss.item():.4f}")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            model.eval()
            print(f"Iteration {i}: Fine-tuning applied due to incorrect solution. (Loss = {loss.item():.4f})")
        else:
            print(f"Iteration {i}: Dreamed solution was correct. No fine-tuning needed.")

        print(f"Iteration {i} completed.\n")

    print("Saving model to ./dreamAI_refined ...")
    model.save_pretrained("./dreamAI_refined")
    print("Model saved to ./dreamAI_refined")

dream_and_improve(iterations=3)

# Optional: Multimodal Fine-Tuning with Unsloth

This optional component demonstrates how you can integrate Unsloth for multimodal fine-tuning. Unsloth is designed to accelerate fine-tuning with reduced VRAM usage while supporting advanced features like 4-bit quantization. In this section, we load a model using Unsloth's API and perform a few fine-tuning steps on a sample multimodal prompt. This component can be enabled as needed.

In [25]:
# Upgrade torch to a compatible version and install unsloth
!pip install torch==2.6.0+cu124 -f https://download.pytorch.org/whl/torch_stable.html > /dev/null 2>&1
!pip install unsloth > /dev/null 2>&1


In [ ]:
import torch
import sys

# Helper: Force monkey-patching function
def force_patch(module, attr_name, dummy):
    print(f"Force patching {attr_name} in {module.__name__}")
    setattr(module, attr_name, dummy)

# Import torch._inductor.utils and force-patch problematic attributes
try:
    import torch._inductor.utils as inductor_utils
except ImportError as e:
    print("Error importing torch._inductor.utils:", e)
    sys.exit(1)

# Ensure upcast_compute_type and boolean_ops are callable (patch if necessary)
if not callable(getattr(inductor_utils, "upcast_compute_type", None)):
    force_patch(inductor_utils, "upcast_compute_type", lambda *args, **kwargs: args[0] if args else None)
else:
    print("upcast_compute_type is callable in torch._inductor.utils")

if not callable(getattr(inductor_utils, "boolean_ops", None)):
    force_patch(inductor_utils, "boolean_ops", lambda *args, **kwargs: None)
else:
    print("boolean_ops is callable in torch._inductor.utils")

# Force-patch ir_dataclass unconditionally to a dummy decorator that returns the class unchanged
force_patch(inductor_utils, "ir_dataclass", lambda *args, **kwargs: (lambda cls: cls))

# Import torch._dynamo.utils and ensure set_feature_use is callable
try:
    import torch._dynamo.utils as dynamo_utils
    if not callable(getattr(dynamo_utils, "set_feature_use", None)):
        force_patch(dynamo_utils, "set_feature_use", lambda *args, **kwargs: None)
    else:
        print("set_feature_use is callable in torch._dynamo.utils")
except ImportError as e:
    print("Error importing torch._dynamo.utils:", e)
    sys.exit(1)

# Attempt to import Unsloth with enhanced error handling
try:
    from unsloth import FastLanguageModel as UnslothFastLM
    print("Unsloth imported successfully.")
except ImportError as e:
    print("Error importing Unsloth:", e)
    print("Please ensure that torch is upgraded to a compatible version (e.g., torch 2.6.0+cu124) and restart the runtime.")
    sys.exit(1)

# Ensure 'device' is defined
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the previously fine-tuned model (saved at "./dreamAI_refined") for further fine-tuning with Unsloth.
try:
    unsloth_model, unsloth_tokenizer = UnslothFastLM.from_pretrained(
        model_name="./dreamAI_refined",  # Use the locally saved model directory
        max_seq_length=2048,
        load_in_4bit=True
    )
    print("Model loaded successfully with Unsloth.")
except Exception as e:
    print("Error loading model with Unsloth:", e)
    sys.exit(1)

# Sample prompt including a marker for an image block (<IMG> ... </IMG>)
sample_prompt = "Describe a complex scenario with both text and visual elements. <IMG>200 201 202 203 204</IMG> <EOR>"
try:
    input_ids_unsloth = unsloth_tokenizer.encode(sample_prompt, return_tensors="pt").to(device)
except Exception as e:
    print("Error encoding sample prompt:", e)
    sys.exit(1)

# Fine-tuning loop using Unsloth's API (simulate a few training steps)
unsloth_model.train()
optimizer_unsloth = torch.optim.Adam(unsloth_model.parameters(), lr=1e-5)
for step in range(3):
    try:
        outputs = unsloth_model(input_ids=input_ids_unsloth, labels=input_ids_unsloth)
        loss = outputs.loss
        optimizer_unsloth.zero_grad()
        loss.backward()
        optimizer_unsloth.step()
        print(f"Step {step}: Loss = {loss.item():.4f}")
    except Exception as e:
        print(f"Error during fine-tuning at step {step}:", e)
        sys.exit(1)
unsloth_model.eval()
print("Optional multimodal fine-tuning with Unsloth completed.")


# Evaluator CLI

This cell implements a complete command‑line evaluator that uses `argparse` to accept a problem description, generates a chain‑of‑thought using MVoT, and (if the problem is a maze) runs GSPO to validate the path. This evaluator can be run from the command line.

In [11]:
import argparse
import sys

# Define generate_maze if not already defined
def generate_maze(rows, cols, wall_prob):
    """
    Generate a random maze represented as a grid (list of lists) where:
      - 0 indicates a free cell,
      - 1 indicates a wall.
    wall_prob is the probability of a cell being a wall.
    The start (0,0) and goal (rows-1, cols-1) cells are always free.
    """
    import random
    maze = []
    for i in range(rows):
        row = []
        for j in range(cols):
            cell = 1 if random.random() < wall_prob else 0
            row.append(cell)
        maze.append(row)
    maze[0][0] = 0
    maze[rows-1][cols-1] = 0
    return maze

# Assume find_path_with_astar and is_path_valid are defined elsewhere
def evaluator_cli():
    parser = argparse.ArgumentParser(description="DREAM AI Evaluator CLI")
    parser.add_argument("--problem", type=str, required=True, help="Problem description to evaluate")
    args = parser.parse_args()
    problem = args.problem
    print("\nEvaluating problem:", problem)

    # Generate chain-of-thought
    chain = generate_mot_trace(problem, max_new_tokens=50)
    print("\nGenerated Chain-of-Thought:\n", chain)

    # If the problem is a maze, run the GSPO evaluation
    if "maze" in problem.lower():
        maze = generate_maze(5, 5, 0.3)
        path = find_path_with_astar(maze, (0, 0), (4, 4))
        if is_path_valid(maze, path):
            print("\nMaze Evaluation: Valid path found:", path)
        else:
            print("\nMaze Evaluation: No valid path found.")
    else:
        print("\nNo additional evaluation available for this problem.")

if __name__ == '__main__':
    # For testing in Colab, simulate command-line arguments:
    sys.argv = ["colab_kernel_launcher.py", "--problem", "Test maze problem: solve a 5x5 maze with entrance at top-left and exit at bottom-right"]
    evaluator_cli()



Evaluating problem: Test maze problem: solve a 5x5 maze with entrance at top-left and exit at bottom-right

Generated Chain-of-Thought:
 Test maze problem: solve a 5x5 maze with entrance at top-left and exit at bottom-right.

Here is the maze:
```
  1 | 2 | 3
  ---------
  4 | 5 | 6
  ---

Maze Evaluation: Valid path found: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]


# Gradio UI (Interactive Explorer)

This cell sets up a Gradio interface so you can interactively enter a problem description and see the generated chain‑of‑thought (with simulated visual descriptions).

In [18]:
import gradio as gr

with gr.Blocks(title="DREAM AI Advanced Management UI") as demo:
    with gr.Tabs():
        with gr.Tab("Problem Reasoning"):
            with gr.Row():
                problem_input = gr.Textbox(lines=3, label="Problem Description", placeholder="Enter your problem description here...")
                mode_dropdown = gr.Dropdown(choices=["maze", "general", "custom"], value="general", label="Reasoning Mode")
            with gr.Row():
                max_tokens_slider = gr.Slider(minimum=10, maximum=200, step=1, value=64, label="Max New Tokens")
                debug_checkbox = gr.Checkbox(label="Enable Debug Mode", value=False)
                temp_slider = gr.Slider(minimum=0.0, maximum=1.0, step=0.05, value=0.8, label="Temperature")
            run_button = gr.Button("Run Reasoning")
            reasoning_output = gr.Markdown(label="Reasoning Output")

            # Connect components to simulate reasoning (replace with your actual function)
            run_button.click(fn=lambda problem, mode, tokens, debug, temp: (
                f"**Problem:** {problem}\n\n"
                f"**Mode:** {mode}\n\n"
                f"**Max Tokens:** {tokens}\n\n"
                f"**Temperature:** {temp}\n\n"
                f"**Debug:** {'Enabled' if debug else 'Disabled'}\n\n"
                "1. Received input.\n"
                "2. Processing reasoning...\n"
                "3. Final output generated."
            ), inputs=[problem_input, mode_dropdown, max_tokens_slider, debug_checkbox, temp_slider], outputs=reasoning_output)

        with gr.Tab("Model Configuration"):
            lr_slider = gr.Slider(minimum=1e-6, maximum=1e-3, value=1e-5, label="Learning Rate", step=1e-6)
            model_type_dropdown = gr.Dropdown(choices=["Chameleon‑7B", "Other"], value="Chameleon‑7B", label="Model Type")
            config_mode_dropdown = gr.Dropdown(choices=["Training", "Inference", "Evaluation"], value="Inference", label="Operation Mode")
            save_button = gr.Button("Save Model")
            load_button = gr.Button("Load Model")
            config_output = gr.Markdown(label="Model Configuration Output")

            # Dummy functions for saving and loading models
            save_button.click(fn=lambda lr, mtype, op: f"Model saved with LR={lr}, Type={mtype}, Mode={op}.",
                              inputs=[lr_slider, model_type_dropdown, config_mode_dropdown],
                              outputs=config_output)
            load_button.click(fn=lambda: "Model loaded successfully.",
                              inputs=[], outputs=config_output)

        with gr.Tab("Dataset Upload"):
            dataset_uploader = gr.File(label="Upload Dataset (CSV, JSON, etc.)")
            dataset_info = gr.Markdown(label="Dataset Info")

            # Dummy function to display dataset info
            dataset_uploader.change(fn=lambda file: f"Uploaded file: {file.name}\nSize: {file.size} bytes",
                                    inputs=dataset_uploader, outputs=dataset_info)

        with gr.Tab("Documentation"):
            documentation_md = gr.Markdown(value="""
            # DREAM AI Documentation
            **Overview:**
            The DREAM AI system integrates multimodal chain-of-thought generation, guided symbolic optimization (GSPO), recursive self‑optimization (dreaming), and advanced model management.

            **Tabs Overview:**
            - **Problem Reasoning:** Input a problem and run the reasoning engine.
            - **Model Configuration:** Adjust model parameters and manage saving/loading.
            - **Dataset Upload:** Upload datasets and view file information.
            - **Documentation:** Read usage instructions and system details.
            """, label="Documentation")

demo.launch(debug=False, share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://106271943c9f10d51a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# DREAM AI Multisensory Processing

In [ ]:
import gradio as gr

def process_multimodal_inputs(text_input, image_input, audio_input):
    output = f"**Text Input:** {text_input}\n\n"
    if image_input is not None:
        output += "**Image Provided:** Yes\n\n"
    else:
        output += "**Image Provided:** No\n\n"
    if audio_input is not None:
        # Gradio's Audio input (with type="numpy") returns a numpy array representing the waveform.
        output += "**Audio Provided:** Yes\n\n"
    else:
        output += "**Audio Provided:** No\n\n"
    # Simulate combined processing of sensory inputs:
    output += "Processing sensory inputs...\n"
    output += "- Text analysis complete.\n"
    output += "- Image features extracted.\n"
    output += "- Audio transcribed (simulation).\n"
    output += "\n**Final Combined Output:** All modalities processed successfully."
    return output

with gr.Blocks(title="DREAM AI Multisensory Processing") as demo:
    gr.Markdown("# DREAM AI Multisensory Input Management")
    gr.Markdown("This interface demonstrates processing of multiple sensory inputs including text, image, and audio.")

    with gr.Tabs():
        with gr.Tab("Text & Image"):
            text_input = gr.Textbox(lines=3, label="Enter Text")
            image_input = gr.Image(label="Upload an Image", type="numpy")
            btn_text_image = gr.Button("Process Text & Image")
            output_text_image = gr.Markdown(label="Output")
            btn_text_image.click(
                fn=lambda txt, img: f"**Processed Text:** {txt}\n**Image:** {'Provided' if img is not None else 'Not Provided'}",
                inputs=[text_input, image_input],
                outputs=output_text_image
            )

        with gr.Tab("Audio"):
            audio_input = gr.Audio(label="Upload Audio", type="numpy")
            btn_audio = gr.Button("Process Audio")
            output_audio = gr.Markdown(label="Output")
            btn_audio.click(
                fn=lambda audio: f"**Audio Processing:** {'Audio data received' if audio is not None else 'No audio provided'}",
                inputs=audio_input,
                outputs=output_audio
            )

        with gr.Tab("Combined Sensory Input"):
            text_input2 = gr.Textbox(lines=3, label="Enter Text")
            image_input2 = gr.Image(label="Upload an Image", type="numpy")
            audio_input2 = gr.Audio(label="Upload Audio", type="numpy")
            btn_combined = gr.Button("Process All Inputs")
            output_combined = gr.Markdown(label="Combined Output")
            btn_combined.click(
                fn=process_multimodal_inputs,
                inputs=[text_input2, image_input2, audio_input2],
                outputs=output_combined
            )

        with gr.Tab("Documentation"):
            doc_md = gr.Markdown(value="""
            # DREAM AI Multisensory Processing Documentation

            **Overview:**
            This interface demonstrates how to process multiple sensory inputs:

            - **Text Input:** Type in your textual prompt.
            - **Image Input:** Upload an image file.
            - **Audio Input:** Upload an audio file.

            **Tabs:**
            - **Text & Image:** Processes text and image separately.
            - **Audio:** Processes audio input alone.
            - **Combined Sensory Input:** Processes text, image, and audio together.
            - **Documentation:** Provides guidance on how to use the interface.

            The outputs are simulated and provide a basic report of the received inputs.
            """, label="Documentation")

demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://de5c2f55a103f39464.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Conclusion

This notebook has implemented a complete D.R.E.A.M. AI system with:

- Multimodal chain-of-thought generation (MVoT) that unifies text and image tokens.
- A guided symbolic optimizer (GSPO) that uses A* for maze planning.
- A recursive self-optimization loop (dreaming) for on-the-fly fine-tuning.
- An optional multimodal fine-tuning component using Unsloth.
- An evaluator CLI to run evaluations from the command line.
- An interactive Gradio UI for exploring reasoning traces.

Feel free to further refine each component for your specific application or domain.